# 🧮 Computational Theory - 2025 Assessment
## 📁 `tasks.ipynb` - Computational Theory Notebook

**Module:** Computational Theory  
**Year:** 2025  
**Author:** *James Doonan*  
**Repository:** *https://github.com/JamesDoonan1/computational_theory*  
**Submission Deadline:** 🗓 **Sunday, 4 May 2025**  

---

## 📜 **Assessment Overview**
This Jupyter Notebook contains solutions to the **Computational Theory** assessment tasks. Each task is clearly labeled, documented, and implemented according to the **module requirements**.

### 📑 **Contents**
🔹 [Task 1: Binary Representations](#task-1-binary-representations)  
🔹 [Task 2: Hash Functions](#task-2-hash-functions)  
🔹 [Task 3: SHA256 Padding](#task-3-sha256-padding)  
🔹 [Task 4: Prime Numbers](#task-4-prime-numbers)  
🔹 [Task 5: Roots](#task-5-roots)  
🔹 [Task 6: Proof of Work](#task-6-proof-of-work)  
🔹 [Task 7: Turing Machine](#task-7-turing-machine)  
🔹 [Task 8: Computational Complexity](#task-8-computational-complexity)  

---

## ⚡ **Instructions for this Notebook**
- Each **task** is implemented in a separate section.
- The notebook follows a **structured format**:
  - **📌 Task Introduction:** Description of the problem.
  - **📝 Code Implementation:** Python solutions with explanations.
  - **🛠️ Testing & Validation:** Demonstrating correctness.
- Code follows **PEP8 standards** for readability.
- Markdown cells provide **explanations, research, and insights**.

---


# 🛠 Task 1: Binary Representations
## 🔹 Rotations, Choice, and Majority Functions

This task implements **bitwise operations** on **32-bit unsigned integers** commonly used in **cryptography**.

### ✅ Functions Implemented:
1. **`rotl(x, n=1)`** – Rotate bits **left**.
2. **`rotr(x, n=1)`** – Rotate bits **right**.
3. **`ch(x, y, z)`** – Bitwise choice function.
4. **`maj(x, y, z)`** – Bitwise majority function.

### 🔹 Key Considerations:
- Operate within **32-bit unsigned integer space**.
- **Use bitwise operators** (`&`, `|`, `^`, `~`).
- Ensure results **wrap around correctly** using `& 0xFFFFFFFF`.

---


In [16]:
import os
import struct
import random
import math
import numpy as np
import hashlib
import nltk
from nltk.corpus import words

In [2]:
# Function: Rotate left (ROTL)
def rotl(x: int, n: int =1) -> int:
    """
    Rotates a 32-bit integer left by n positions.  

    Parameters:
    x (int) : 32-bit integer to rotate  
    n (int) : Number of positions to rotate left (default 1)  

    Returns:
    int : 32-bit integer rotated left by n positions

    """
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

# Example Usage  
example_value = 0x12345678 # example value to rotate
rot1_result = rotl(example_value, 4) # rotate left by 4 positions
print(f"ROTL(0x12345678, 4) -> {rot1_result:#010X}") # expected result: 0x23456781


ROTL(0x12345678, 4) -> 0X23456781


In [3]:
# Function: Rotate right (ROTR)
def rotr(x: int, n: int =1) -> int:
    """
    Rotates a 32-bit integer right by n positions.  

    Parameters:
    x (int) : 32-bit integer to rotate  
    n (int) : Number of positions to rotate right (default: 1)  

    Returns:
    int : 32-bit integer rotated right by n positions

    """
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF 

# Example Usage  
example_value = 0x12345678 # example value to rotate
rotr_result = rotr(example_value, 4) # rotate left by 4 positions
print(f"ROTR(0x12345678, 4) -> {rotr_result:#010X}") # expected result: 0x23456781


ROTR(0x12345678, 4) -> 0X81234567


In [4]:
def ch(x: int, y: int, z: int) -> int:
    """
    Choice bits from y where x has bits set to 1,
    and bits from z whee x has bits set to 0.  

    Parameters:
    x (int) : Control bits.  
    y (int) : First input.  
    z (int) : Second input.  

    Returns:
    int : The result of the choice operation.  

    """
    return (x & y) ^ (~x & z) # If x is 1, take y, otherwise take z

x = 0b10101010101010101010101010101010  # Binary control mask
y = 0b11110000111100001111000011110000  # Option 1
z = 0b00001111000011110000111100001111  # Option 2

ch_result = ch(x, y, z)

print(f"CH(x, y, z) -> {ch_result:010x}") # expected output: 00a5a5a5a5

CH(x, y, z) -> 00a5a5a5a5


In [5]:
def maj(x: int, y: int, z: int) -> int:
    """
    Majority bits from x, y, and z.  

    Parameters:
    x (int) : First input.  
    y (int) : Second input.  
    z (int) : Third input.  

    Returns:
    int : The result of the majority operation.  

    """
    return (x & y) ^ (x & z) ^ (y & z) # Majority of bits

# ✅ Example Usage
x = 0b10101010101010101010101010101010  # Example binary value
y = 0b11110000111100001111000011110000  # Example binary value
z = 0b00001111000011110000111100001111  # Example binary value

maj_result = maj(x, y, z)

# Print results in hexadecimal format
print(f"MAJ(x, y, z) -> {maj_result:#010x}") # expected output varies based on inputs


MAJ(x, y, z) -> 0xaaaaaaaa


# Task 2 Hash Functions  

## Problem Statement  
Convert the following **C-based hash function** into python, test it and analyze why **31 and *101 were used as constants.  

### Given C Code:
```c
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


In [6]:
def kr_hash(s: str) -> int:
    """
    Computes a hash for a given string using the method from 
    Kernighan and Ritchie's "The C Programming Language".  

    Parameters:
    s (str) : The input string.

    Returns:
    int : The computed hash value.
    """  
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval # Compute hash using ASCII values
    return hashval % 101 # Return hash value modulo 101

# Example Usage & Test
test_strings = ["hello", "world", "computational", "theory", "openai"]
for s in test_strings:
    print(f"Hash'({s}) -> {kr_hash(s)}") 


Hash'(hello) -> 17
Hash'(world) -> 34
Hash'(computational) -> 42
Hash'(theory) -> 77
Hash'(openai) -> 35


## Why use `31` and `101` ?  
The hash function in the C code follow a common pattern in hashing, using **prime numbers** to reduce collisions.  

### **Why `31`?** 
- **Prime number**: Helps distribute hash value **evenly** across a range.  
- **Efficient multiplication**:  `31` can be computed as ``(x << 5) - x`` (bit shift and subtraction).  
- **Commonly used in string hashing** (e.g., Java uses `31` in it's `hashCode()` function). 

### **Why `101`?**  
- **Prime modulus**: Helps ensure a more uniform distribution of hash values.  
- **Prevents excessive collisions**: if `101` were a power of 2, the hash function might cause clustering.  

Thus `31` and `101` work together to create an **efficient and well-distributed** hash function.

In [7]:
# Test Case Validation for task 2. 
def test_kr_hash():
    """
    Run multiple test cases to verify the correctness of the hash function.  
    """
    test_cases = {
        "hello": kr_hash("hello"),
        "world": kr_hash("world"),
        "computational": kr_hash("computational"),
        "theory": kr_hash("theory"),
        "openai": kr_hash("openai")
    }
    for key, expected in test_cases.items():
        result = kr_hash(key)
        assert result == expected, f"Test Failed for: {key}: got {result}, expected {expected}"
        print(f"✅ Test passed for '{key}' -> Hash: {result}")

# Run the test cases
test_kr_hash() # All tests pass

✅ Test passed for 'hello' -> Hash: 17
✅ Test passed for 'world' -> Hash: 34
✅ Test passed for 'computational' -> Hash: 42
✅ Test passed for 'theory' -> Hash: 77
✅ Test passed for 'openai' -> Hash: 35


# 🛠 Task 3: SHA256 Padding
## 🔹 Cryptographic padding in SHA256

This task implements **SHAS256 passing scheme** which is a critical step in the SHA256 hashing algorithm. The padding makes sure that the input message meets the requirements for processing in 512-bit blocks.

### ✅ Problem Statement:  
Write a Python function that calculates the SHA256 padding for a given file.  
The function should:
1. Append a single **1** bit to the message.
2. Append enough **0** bits so that the length of the padded message is congruent to 448 modulo 512.
3. Append the original length of the message (in bits) as a 64-bit big-endian integer.
4. Print the padding in hexadecimal format.

### 🔹 Key Considerations:
- The input file is read in **binary mode** to handle all types of files (text, images, etc.).
- The padding must ensure that the total length of the message (including padding) is a multiple of 512 bits.
- The original length of the message is appended as a **64-bit big-endian integer**.  

---


In [8]:
def sha256_padding(file_path: str):
    """
    Calculate the SHA256 padding for a given file.

    Parameters:
    file_path (str): Path to the input file.

    Returns:
    str: The padding in hexadecimal format.
    """
    # Step 1: Read the file as binary data
    with open(file_path, "rb") as file:
        message = file.read()

    # Step 2: Calculate the original length of the data in bits
    original_length = len(message) * 8

    # Step 3: Append a single '1' bit to the message (0x80 in hex) 
    padded_message = message + b'\x80'

    # Step 4: Append '0' bits until the length in bits is 448 (mod 512)
    padding_length = (448 - (original_length + 8) % 512) % 512
    padded_message += b'\x00' * (padding_length // 8)

    # Step 5: Append the original length of the message in bits as a 64-bit big-endian integer
    padded_message += struct.pack('>Q', original_length) 

    # Step 6: Return the padded message in hexadecimal format
    padding = padded_message[len(message):]
    return padding.hex()

# Test the function with a sample file
# Create a sample file if it does not exist
file_path = "sample.txt"

if not os.path.exists(file_path):
    with open(file_path, "wb") as file:
        file.write(b"abc") # Binary mode ensures correct encoding

# Calculate the SHA256 padding
padding_hex = sha256_padding(file_path)

# Format the hex string into space-separated bytes (2 characters each)
formatted_padding = " ".join(padding_hex[i:i+2] for i in range(0, len(padding_hex), 2))

# Print in the expected format
print(f"SHA256 Padding for '{file_path}':\n{formatted_padding}")



SHA256 Padding for 'sample.txt':
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


# 🛠 Task 4: Prime Numbers  
## 🔹 Calculating the First 100 Prime Numbers  

This task involves calculating the first 100 prime numbers using **two different algorithms**:  
1. **Miller-Rabin Primality Test** – A probabilistic test that is highly efficient for large numbers.   
2. **Trial Division with Square Root Optimisation** – A deterministic method that checks divisibility up to the square root of a number. 

### ✅ Problem Statement  
Calculate the first 100 prime numbers using two different algorithms. Explain how the algorithms work and compare their performance.  

---  

### 🔹 Algorithm 1: Miller-Rabin Primality Test  

### Justification of choice  
The **Miller-Rabin primality test** is a probabilistic algorithm to determine if a number is prime. It is based on the properties of **strong pseudoprimes** and is highly efficient for large numbers. Although it is used primarily with large prime numbers, I found the algorithm pretty interesting and I also felt it was different to standard algorithms. I'm aware it is a probabilistic algorithm which means it is not ideal for generating the first 100 prime numbers because it is a **probabilistic** test, not a **deterministic** method for listing primes. I know  it is not designed for sequential prime generation. I was interested to see if I could get it working. 

#### Steps:  
1. **Decompose `n`:** Write `n` as `d * 2^r + 1`, where `d` is odd.  
2. **Choose Bases:** Select a set of bases (e.g., 2, 3, 5, 7, etc.) for testing.  
3. **Check Composite:** For each base, check if it satisfies certain conditions that imply `n` is composite.  
4. **Repeat:** Perform the test `k` times to reduce the probability of error.  

#### Advantages:  
- **Efficient:** Works well for large numbers.  
- **Probabilistic:** Can be made deterministic for numbers less than 2^64.  

#### Disadvantages:  
- **Probabilistic:** Small chance of false positives (though negligible for practical purposes).  
---  


#### 🛠 Function Description
####  `miller_rabin(n, k=5)`

#### 🔹 Purpose
This function performs the **Miller-Rabin primality test** to determine if a number `n` is prime. It’s a probabilistic test but can be made deterministic for numbers less than 2^64.

##### 🔹 Steps
1. **Handle Edge Cases:**
   - If `n < 2`, return `False` (not prime).
   - If `n` is 2 or 3, return `True` (prime).
   - If `n` is even, return `False` (not prime).

2. **Decompose `n`:**
   - Write `n` as `n = 2^r * d + 1`, where `d` is odd.

3. **Choose Bases:**
   - For numbers ≤ 2^64, use a deterministic set of bases (2, 3, 5, etc.).
   - For larger numbers, use `k` random bases.

4. **Check Composite:**
   - For each base, check if `n` is composite using the `check_composite` function.
   - If `n` is composite for any base, return `False`.
   - If `n` passes all tests, return `True` (likely prime).

In [9]:
def miller_rabin(n, k=5):
    """
    Perform the Miller-Rabin primality test on n with k iterations.
    This is a probabilistic test but can be made deterministic for small numbers.
    """
    if n < 2:
        return False
    if n in {2, 3}:
        return True
    if n % 2 == 0:
        return False

    # Write n as 2^r * d + 1 with d odd
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # Deterministic bases for numbers <= 2^64
    bases = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41]
    if n < 2**64:
        bases = [b for b in bases if b < n]
    else:
        bases = [random.sample(range(2, n-1), k)] # Random bases for large numbers

    def check_composite(a, d, n, r):
        """
        Check if n is composite using the Miller-Rabin test with witness a.
        """
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            return False
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                return False
        return True
    
    for base in bases:
                if check_composite(base, d, n, r):
                    return False
    return True
        

#### 🛠 `optimised_miller_rabin_primes(count)`

#### 🔹 Purpose
This function generates the first `count` prime numbers using the **Miller-Rabin primality test**...

In [10]:
def optimised_miller_rabin_primes(count):
    """
    Generate a list of prime numbers using the Miller-Rabin primality test.
    """
    primes = []
    num = 2  # Start checking from the first prime
    
    while len(primes) < count:
        if miller_rabin(num):
            primes.append(num)
        num += 1 if num == 2 else 2  # Skip even numbers after 2
    
    return primes[:count]

In [11]:
# Generate the first 100 prime numbers
first_100_primes = optimised_miller_rabin_primes(100)
print("First 100 prime numbers:")
print(first_100_primes)

First 100 prime numbers:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


### 🔹 Algorithm 2: Square Root Optimisation

### Justification of choice 
The **square root optimisation** is a deterministic method for checking primality. It works by testing divisibility up to the square root of the number. I chose this for its simplicity and efficiency. It provides a definite answer (no probabilistic uncertainty) and is efficient for small numbers. **In our notes** you can see this:

 
**Square root test for primality.**  
```
def isprime_sqrt(n):  
        Test whether n is prime.    
        Loop through 2...floor(sqrt(n)).  
        for i in range(2, np.floor(np.sqrt(n)).astype(int) + 1):   
            # Calculate remainder of n divided by i.  
            if n % i == 0:  
                If this is zero, then n is not prime.  
                return False  
        If we get here, then n is prime.   
        return True   
``` 
 

This uses the concept of optimising primality testing by checking divisibility only up to the **square root of a number** It is based on the observation that if a number `n` is composite, at least one of it's actors must be less than or equal to `√n`.  
So this **influenced my choice** because of:  
1. The efficiency of how fast this algorithm is.  
2. The description that "a small factor is balanced by a large factor, and the balance point is the square root"  
3.The algorithm is straight forward to implement and understand    

**How I adapted it**  
1. Skipping Even Numbers After 2 – Since all even numbers (except 2) are non-prime, we skip them, reducing unnecessary computations.    
2. More Efficient Prime Generation – When generating the first n primes, we check divisibility only for previously found primes, avoiding redundant checks  
3. Refactored Code for Clarity – The updated function ensures readability while maintaining efficiency.  

**The reasoning behind the square root approach can be visualised in our notes:**

** FROM THE NOTES - Consider this product of two primes - note that it is not prime.**  

``` 
    n = 17 * 19   
    The square root of n.  
    np.sqrt(n)  
    Output: 17.972

    Try all numbers up to the square root.  
    for i in range(2, int(np.sqrt(n)) + 1):  
        if n % i == 0:  
            print(f"{n} = {i} * {n // i}")  Factorization check  
    Output: 323 = 17 * 19 
 ``` 

Source:  https://github.com/ianmcloughlin/computational_theory/blob/main/materials/prime_numbers.ipynb  

#### Square Root Optimisation Steps:  
1. **Check Divisibility:** For each number `n`, check if it is divisible by any integer from 2 to `√n`.  
2. **Skip Even Numbers:** After checking 2, skip all even numbers to improve efficiency.  

#### Advantages:  
- **Deterministic:** Always provides a correct result.  
- **Simple:** Easy to implement and understand.  

#### Disadvantages:  
- **Inefficient for Large Numbers:** Time complexity is O(√n), making it slow for very large inputs.  

---

In [12]:
def isprime_sqrt(n: int) -> bool:
    """
    Test whether n is prime using the square root optimization.

    Parameters:
    n (int): The number to check for primality.

    Returns:
    bool: True if the number is prime, False otherwise.
    """  
    if n < 2:
        return False
    for i in range(2,int(np.sqrt(n))+1):
        if n % i == 0:
            return False
    return True

In [13]:
def first_n_primes(n: int) -> list[int]:
    """
    Find the first n prime numbers using an optimized approach.

    Parameters:
    n (int): The number of primes to generate.

    Returns:
    list[int]: A list of the first n prime numbers.
    """
    primes = []
    num = 2
    while len(primes) < n:
        if isprime_sqrt(num):
            primes.append(num)
        num += 1 if num == 2 else 2 # Skip even numbers after 2
    return primes

n = 100
primes = first_n_primes(n)
print(f"First {n} primes: {primes}")


First 100 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


# 🛠 Task 5: Roots  

## 🔹 Square Roots in Cryptography  

this task implements the computationa of the **first 32 bits of the fractional part of the square roots** of the **first 100 prime numbers**. These values are widely uside in cryptographic has functions as **SHA-256**, when they serve as initalization constants.  

### ✅ Problem Statement:  
Write a Python function that calculates the **first 32 bits of the fractional part** of the **square root of the first 100 prime numbers**.  

The function should:  
1. **Use the `first_n_primes(n)` function from Task 4** to get the first 100 prime numbers.    
2. **Compute the square root of each prime**.  
3. **Extract the fractional part** (i.e., the part after the decimal point).  
4. **Multiply by `2³²` (4294967296) and convert to an integer**.  
5. **Print the values in hexadecimal format** for better readability.  


### 🔹 Key Considerations:  
- The **first 100 primes are reused from Task 4**, ensuring efficiency.    
- The **fractional part** is extracted using standard floating-point operations.  
- The **multiplication  by `2³²` ensures a 32-bit integer representation**.  
- The results are printed in **hexadecimal format** for clarity and comparison with cryptographic constants.  

---


In [ ]:
import math  # Required for square root calculations

def fractional_root_bits(prime: int) -> int:
    """
    Compute the first 32 bits of the fractional part of the square root of a prime number.

    Parameters:
    prime (int): The prime number.

    Returns:
    int: The 32-bit integer representation of the fractional part.
    """
    sqrt_val = math.sqrt(prime)
    fractional_part = sqrt_val - math.floor(sqrt_val)  # Extract fractional part
    result = int(fractional_part * (2**32))  # Convert to 32-bit integer
    return result

# ✅ Directly use `first_n_primes(n)` from Task 4
fractional_bits = {p: fractional_root_bits(p) for p in primes}  # `primes` is from Task 4

# ✅ Display results
print("\n🔹 First 32 bits of fractional part of square roots (in hex):\n")
for prime, frac_bits in fractional_bits.items():
    print(f"Prime: {prime}, Fractional Bits (Hex): {frac_bits:08x}")  # Print in hex



🔹 First 32 bits of fractional part of square roots (in hex):

Prime: 2, Fractional Bits (Hex): 6a09e667
Prime: 3, Fractional Bits (Hex): bb67ae85
Prime: 5, Fractional Bits (Hex): 3c6ef372
Prime: 7, Fractional Bits (Hex): a54ff53a
Prime: 11, Fractional Bits (Hex): 510e527f
Prime: 13, Fractional Bits (Hex): 9b05688c
Prime: 17, Fractional Bits (Hex): 1f83d9ab
Prime: 19, Fractional Bits (Hex): 5be0cd19
Prime: 23, Fractional Bits (Hex): cbbb9d5d
Prime: 29, Fractional Bits (Hex): 629a292a
Prime: 31, Fractional Bits (Hex): 9159015a
Prime: 37, Fractional Bits (Hex): 152fecd8
Prime: 41, Fractional Bits (Hex): 67332667
Prime: 43, Fractional Bits (Hex): 8eb44a87
Prime: 47, Fractional Bits (Hex): db0c2e0d
Prime: 53, Fractional Bits (Hex): 47b5481d
Prime: 59, Fractional Bits (Hex): ae5f9156
Prime: 61, Fractional Bits (Hex): cf6c85d3
Prime: 67, Fractional Bits (Hex): 2f73477d
Prime: 71, Fractional Bits (Hex): 6d1826ca
Prime: 73, Fractional Bits (Hex): 8b43d457
Prime: 79, Fractional Bits (Hex): e360

# 🛠 Task 6: Proof of Work  

## 🔹 Finding Words with Leading Zero Bits in SHA-256  

This task implements a **Proof-of-Work style challenge**, where I **search for words in the English language that produce SHA-256 hashes with the highest number of leading `0` bits**.  

This concept is **related to blockchain mining**, where finding a hash with leading zeroes is required for validating blocks. In this case, instead of mining, searching for English words that naturally have **SHA-256 hashes with many leading zeroes**.  

### ✅ Problem Statement  
Write a Python function that:  
1. **Uses a dataset of English words**. 
2. **Computes the SHA-256 hash** of each word.  
3. **Counts the number of leading `0` bits** in the hash.  
4. **Finds the word(s) with the most leading `0` bits**.  
5. **Verifies that the words exist in at least one English dictionary**.  
6. **Prints the top words along with their SHA-256 hashes**.  

### 🔹 Key Considerations  
- **SHA-256 hashes are computed using Python’s `hashlib` module**.  
- **Words are retrieved from an English dictionary dataset**.  
- **The number of leading `0` bits is counted in the hash’s binary representation**.  
- **The output will display words with the highest leading zero count, their bit count, and their SHA-256 hash**.  

---


In [17]:
nltk.download('words')

def sha256_leading_zeros(word: str) -> int:
    """
    Compute the SHA-256 hash of a word and count the number of leading zero bits.

    Parameters:
    word (str): The input word.

    Returns:
    int: The count of leading zero bits in the hash.
    """

    hash_hex = hashlib.sha256(word.encode()).hexdigest() # Compute SHA-256 hash
    hash_bin = bin(int(hash_hex, 16))[2:].zfill(256)  # Convert to binary and pad to 256 bits
    return len(hash_bin) - len(hash_bin.lstrip('0'))  # Count leading zeros

# Step 1: Get a list of English words
english_words = words.words()

# Step 2: Compute leading zero bits for each word
word_hashes = {word: sha256_leading_zeros(word) for word in english_words}

# Step 3: Find the word with the most leading zeros
max_zeros = max(word_hashes.values())
best_words = [word for word, zeros in word_hashes.items() if zeros == max_zeros]

# Step 4: Display the results
print(f"Word(s) with the most leading zeros ({max_zeros} zeros):")
for word in best_words:
    print(f"Word: {word}, Leading Zeros: {max_zeros}, SHA-256: {hashlib.sha256(word.encode()).hexdigest()}")

[nltk_data] Downloading package words to C:\Users\James
[nltk_data]     Doonan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


Word(s) with the most leading zeros (16 zeros):
Word: guilefulness, Leading Zeros: 16, SHA-256: 0000d79e1c6964e6806e9bbdaaaecb63dfabdb498f72bf28944119de1fe90d63
Word: mismatchment, Leading Zeros: 16, SHA-256: 0000bb6ede9f29a01d35e15320229aa0fbd73cf8eb8bc0aac80d6a97fba63fee
